# Testing APIs to get historical ansd daily FXrates

In [ ]:
import requests
import pandas as pd
from datetime import date
import json

## 1. exchangerate.host --> FX-re jo lehet, ingyen tier lefedi teljesen ami kell
### Free api with 100 monthly request
- `email`: your@emailaddress.com
- `pass`: Yourpassword01%
- `API Access Key`: yourapikey
- LINK to Access counter: https://exchangerate.host/dashboard

In [ ]:
# több endpoint van görgess lejebb a historikusert
endpoint = 'live'
access_key = 'yourapikey'
## 100 free request per month
##response = requests.get('https://api.exchangerate.host/' + endpoint + '?access_key=' + access_key)
#data = response.json()

#print(data)

Par mezö jon vissza:
- `success` --> bolean, logikus
- `timestamp` --> epoch time --> hany ms telt el ´70 januar 1 ota
- `source` --> mi a fö currency --> alapbol USD, sztem jo is ez igy
- `quotes` --> egy ujabb nested JSON a valuta parokkal

In [ ]:
print(json.dumps(data, indent=4))

In [ ]:
# dupla key-el el is lehet erni a valutakat
for currency in ['SEK', 'DKK', 'NOK', 'EUR', 'CAD', 'GBP', 'CHF']:
    print(currency, data['quotes'][f'USD{currency}'])

## Historikus endpoint
- Egy fix multbeli datumra kerdezek le adatokat.
- Adatlyukak befoltozasara lehet jo.
### Vigyazz az url szerkesztesnel az elsö argument ?-el jön, majd a többi &-el elövalasztva!

In [ ]:
# ez a historikus endpoint templateje
endpoint = 'historical'
access_key = 'yourapikey'
date = '2025-06-17'

## 100 free request per month
response = requests.get('https://api.exchangerate.host/historical' + '?access_key=' + access_key + '&date=' + date)
    
    #'https://api.exchangerate.host/' + endpoint + '?access_key=' + access_key + '?date=' + date)
data_hist = response.json()

print(data_hist)

Extra key-ek:
- `historical` --> boolean
- `date` --> mikorra vonatkozik az adat

In [ ]:
print(json.dumps(data_hist, indent=4))

## Timeframe endpoint
Igazabol idöintervallumot lehet megadni, ez teljesen jo lesz egy alkalmankenti frissiteshez.
### Fontos:
- van egy `start_date` es egy `end_date` key benne
- van `timeframe` --> boolean mezöje
- a `quotes`-on belul nem egyböl a currency parok jönnek, hanem a historikus datumok az alkey-ek es csak azon belul vannak a FX parok --> quotes --> date --> FXpairs
- max 365 napot fedhet le a timeframe
- `1999-01-01`-töl vannak adatok

In [ ]:
# több endpoint van görgess lejebb a historikusert
endpoint = 'timeframe'
access_key = 'yourapikey'
start_date = '2024-01-01'
end_date = '2024-12-31'

## 100 free request per month
response = requests.get('https://api.exchangerate.host/' +
                        endpoint +
                        '?access_key=' + access_key +
                        '&start_date=' + start_date + 
                        '&end_date=' + str(end_date)
                        )
    
    #'https://api.exchangerate.host/' + endpoint + '?access_key=' + access_key + '?date=' + date)
data_tf = response.json()

print(data_tf)

In [ ]:
with open("../jsons/fx_jsons/usd_other_fx_rates_2024.json", "w", encoding="utf-8") as f:
    json.dump(data_tf, f, indent=4, ensure_ascii=False)

## 1. Alpha Vantage --> NEM
Sok elönye lenne:
- makro adatok
- FX arfolyamok
- economy indicators
- financial statements egy helyen

Mindenkeppen megeri megismerni:
- igazabol nm ajultam el töle
- a financial calendar csak az USA-t fedi le es csv-t ad
- a napi 25 requests nagyon keves
- a FX_DAILY endpoint meg egy rablas, mivel egy requestbe egy FX par megy be --> ezzel ellentetbe az `xchangerate.host`eseten egy requesttel lejön minden (csak USD-re vonatkoztatva de akkor is jobb)

- `email`: your@emailaddress.com
- `API Access Key`: yourapikey
- `Documentation`: https://www.alphavantage.co/documentation/

#### Szabalyok:
- napi maximum 25 requests --> keves lesz
- the data 15 mins delayed

In [ ]:
# az alabbi kodblokk leszedi egy api csv-ben az összes ameriaki ceg jelenteseinek idöpontjat 3, 6 vagy 12 honapra elöre
# nem annyira szimpatikus, de egy mkiset megerhet
import csv
import requests

apikey = yourapikey

CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&horizon=3month&apikey={apikey}'

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    for row in my_list:
        print(row)

In [ ]:
# api request temetö, de ez az Alpha Vantage FX api endpointja
import requests

url = f'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=EUR&to_symbol=SEK&outputsize=full&apikey={apikey}'
r = requests.get(url)
data_av_fx = r.json()

print(data_av_fx)

In [ ]:
print(json.dumps(data_av_fx, indent=4))

# 2. Financial Modeling Prep (FMP)
- napi 250 API requests
- van financial calendar is. --> teljesen hasznalhatatlan, csak azt mutatja milyen szamokat varnak.

- `email` --> your@emailaddress.com
- `api_key` --> yourapikey

# Erdekes lehet az AV calendarjat egyutt hasznalni a FMP statementekkel!!!

In [ ]:
import requests

apikey = yourapikey

url = ("https://financialmodelingprep.com/stable/earnings-calendar?apikey={apikey}")
r = requests.get(url)
data_fmp_calendar = r.json()

print(data_fmp_calendar)

In [ ]:
print(json.dumps(data_fmp_calendar, indent=4))